In [1]:
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")
%load_ext autoreload
%autoreload 2

In [2]:
from generate_robot_instruction import encode_task_generation_prompt, generate_task_data, encode_instruct_prompt, generate_instruction_following_chat_data, post_process_chat_response

### Generate Tasks

In [3]:
generate_task_data(
    output_dir="./gpt4_generation/task_generation_v1/",
    num_tasks_to_generate=100,
    prompt_file_name="./prompts_for_gpt/robot_task_prompt_v1.txt",
    model_name="gpt-4-0314"
)

  0%|          | 0/100 [00:00<?, ?it/s]

### [[{'role': 'user', 'content': "You are serving as a task-generation helper for a given robot environment.\xa0\n\n[Environment Description]\nHere is the information about the environment. The environment is called {Two Block World}\xa0\nThere is a 7DOF Franka robot with a parallel gripper.\xa0\nThere are multiple blocks, {block A}, {block B}, ..., with randomized sizes and density in the environment.\nThe Franka robot has a force sensor on the end effector.\nThe Franka robot is mounted on a table.\nThe blocks are initialized at a random position on the table.\nContents in the {} are the names of the objects in the environment.\n\nThe observation of the robot includes:\nThe position and orientation of both blocks and the end effector of the robot\nThe force sensor on the end effector. \n\n[Your Task]\nCome up with 50 different tasks for the robot to perform.\n\n[Output format]\nThe response should follow the template below:\n### Task {i}: {task description}\nwhere {i} is the task num

  1%|          | 1/100 [01:36<2:39:05, 96.42s/it]

['', ' Task 1: Pick up block A and place it on top of block B.\n', ' Task 2: Place block A and block B side by side, touching each other.\n', ' Task 3: Move block A to the center of the table.\n', ' Task 4: Move block B to the edge of the table without causing it to fall off.*\n', ' Task 5: Stack block A and block B to maximize the height of the two blocks.*\n', ' Task 6: Move the lighter block to the left corner of the table and the heavier block to the right corner of the table.*\n', ' Task 7: Stack both blocks on top of each other and ensure they are aligned.*\n', ' Task 8: Move both blocks at least 30 cm away from each other.*\n', ' Task 9: Move both blocks to the left half of the table, keeping at least 10 cm distance between them.*\n', ' Task 10: Pick up and place the lighter block on top of the heavier block, ensuring they touch at only one point.*\n', ' Task 11: Move block B directly adjacent to block A without the two blocks touching each other.\n', ' Task 12: Move both blocks

100%|██████████| 100/100 [03:02<00:00,  1.83s/it]

['', ' Task 1: Pick up block A and place it on top of block B\n', ' Task 2: Move block B to the opposite corner of the table from its initial position\n', ' Task 3: Stack the blocks in order of smallest to largest, starting with the smallest on the bottom^\n', ' Task 4: Move both blocks to one edge of the table\n', ' Task 5: Lift block A without grasping it, only using the force exerted by the end effector*\n', ' Task 6: Arrange the blocks in alphabetical order from left to right on the table\n', ' Task 7: Place block A on its longest edge*\n', ' Task 8: Rearrange the blocks in order of lightest to heaviest block*\n', ' Task 9: Move block B to the center of the table\n', ' Task 10: Balance block A on top of block B on a single edge*\n', ' Task 11: Stack the blocks with block B on top but not touching the table*\n', ' Task 12: Move block B such that it is within 5 cm of block A\n', ' Task 13: Place both blocks in a vertical orientation*\n', ' Task 14: Construct a tower with three blocks

#### Split task data
Split it into tasks used by instruction generation.

In [10]:
import re, os
import string
import json
def split_json_file(json_file, 
                    output_path="./gpt4_generation/task_generation/", n=2):
  """Splits a JSON file into N JSONL files, where each file contains n elements.

  Args:
    json_file: The path to the JSON file to split.
    n: The number of JSONL files to create.

  Returns:
    The list of paths to the created JSONL files.
  """

  with open(json_file, "r") as f:
    data = json.load(f)
  os.makedirs(output_path, exist_ok=True)
  jsonl_files = []
  for i in range(0, len(data), n):
    jsonl_file = f"{output_path}split_{i // n}.jsonl"
    with open(jsonl_file, "w") as outfile:
      for record in data[i : i + n]:
        json.dump(record, outfile)
        outfile.write("\n")

    jsonl_files.append(jsonl_file)
  return jsonl_files

In [11]:
jsonl_files = split_json_file("./gpt4_generation/task_generation/task_regen.json", n=50)

### Generate COT instruct data

In [5]:
from generate_robot_instruction import generate_instruction_following_chat_data


ret = generate_instruction_following_chat_data(output_dir="./gpt4_generation/cot_full_v3",
                                        instruction_prompt_file="./prompts_for_gpt/robot_cot_prompts_v3.txt",
                                        seed_example_path="./prompts_for_gpt/seeded_example_cot_v3.jsonl",
                                        seed_tasks_path="./gpt4_generation/task_generation/split_0.jsonl",
                                        num_instructions_to_generate=300,
                                        output_file="cot_instrut_regen.json"
                                        )

  0%|          | 0/300 [00:00<?, ?it/s]

### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to simulate the task instructions and corresponding responses happening during task solving.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model performance on the generated responses.\n\n[Environment Description] \nThe environment is called "Block World". \nThere is a 7DOF Franka Panda robot with a parallel gripper, it has a force sensor on the end effector. The robot is mounted on a table.\nThere are multiple blocks on tha table, here we use 2 blocks as example. The blocks are called {block A} and {block B}. \nThe blocks are initialized at a random position on the table. They have randomized weight, size.\nThe observation space (when there are two blocks) is a 26 dimension vector, consisting of:\nbl

  0%|          | 1/300 [02:22<11:50:48, 142.64s/it]

  2%|▏         | 6/300 [02:22<11:38:55, 142.64s/it]
Request 1 took 142.64s
Generated 6 instructions, kept 6 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to simulate the task instructions and corresponding responses happening during task solving.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model performance on the generated responses.\n\n[Environment Description] \nThe environment is called "Block World". \nThere is a 7DOF Franka Panda robot with a parallel gripper, it has a force sensor on the end effector. The robot is mounted on a table.\nThere are multiple blocks on tha table, here we use 2 blocks as example. The blocks are called {block A} and {block B}. \nThe blocks are initialized at a random position on the table. They have

prompts: 100%|██████████| 1/1 [01:00<00:00, 60.59s/it]


...

 18%|█▊        | 54/300 [31:16<1:59:34, 29.17s/it]
Request 12 took 61.33s
Generated 3 instructions, kept 3 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to simulate the task instructions and corresponding responses happening during task solving.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model performance on the generated responses.\n\n[Environment Description] \nThe environment is called "Block World". \nThere is a 7DOF Franka Panda robot with a parallel gripper, it has a force sensor on the end effector. The robot is mounted on a table.\nThere are multiple blocks on tha table, here we use 2 blocks as example. The blocks are called {block A} and {block B}. \nThe blocks are initialized at a random position on the table. They have 

 18%|█▊        | 55/300 [32:47<1:59:54, 29.37s/it]

 19%|█▉        | 58/300 [32:47<1:58:26, 29.37s/it]
Request 13 took 90.67s
Generated 4 instructions, kept 4 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to simulate the task instructions and corresponding responses happening during task solving.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model performance on the generated responses.\n\n[Environment Description] \nThe environment is called "Block World". \nThere is a 7DOF Franka Panda robot with a parallel gripper, it has a force sensor on the end effector. The robot is mounted on a table.\nThere are multiple blocks on tha table, here we use 2 blocks as example. The blocks are called {block A} and {block B}. \nThe blocks are initialized at a random position on the table. They have 

prompts:   0%|          | 0/1 [00:48<?, ?it/s]


KeyboardInterrupt: 

In [4]:
from generate_robot_instruction import generate_instruction_following_chat_data


ret = generate_instruction_following_chat_data(output_dir="./gpt4_generation/cot_full_v3",
                                        instruction_prompt_file="./prompts_for_gpt/robot_cot_prompts_v3.txt",
                                        seed_example_path="./prompts_for_gpt/seeded_example_cot_v3.jsonl",
                                        seed_tasks_path="./gpt4_generation/task_generation/split_1.jsonl",
                                        num_instructions_to_generate=300,
                                        output_file="cot_instrut_regen_2.json"
                                        )

  0%|          | 0/300 [00:00<?, ?it/s]

### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to simulate the task instructions and corresponding responses happening during task solving.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model performance on the generated responses.\n\n[Environment Description] \nThe environment is called "Block World". \nThere is a 7DOF Franka Panda robot with a parallel gripper, it has a force sensor on the end effector. The robot is mounted on a table.\nThere are multiple blocks on tha table, here we use 2 blocks as example. The blocks are called {block A} and {block B}. \nThe blocks are initialized at a random position on the table. They have randomized weight, size.\nThe observation space (when there are two blocks) is a 26 dimension vector, consisting of:\nbl

  0%|          | 1/300 [01:36<8:00:56, 96.51s/it]

  1%|▏         | 4/300 [01:36<7:56:07, 96.51s/it]
Request 1 took 96.51s
Generated 4 instructions, kept 4 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to simulate the task instructions and corresponding responses happening during task solving.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model performance on the generated responses.\n\n[Environment Description] \nThe environment is called "Block World". \nThere is a 7DOF Franka Panda robot with a parallel gripper, it has a force sensor on the end effector. The robot is mounted on a table.\nThere are multiple blocks on tha table, here we use 2 blocks as example. The blocks are called {block A} and {block B}. \nThe blocks are initialized at a random position on the table. They have ra

  2%|▏         | 5/300 [06:23<6:10:30, 75.36s/it]

...

 98%|█████████▊| 293/300 [2:36:28<03:27, 29.64s/it]
Request 74 took 69.47s
Generated 1 instructions, kept 1 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to simulate the task instructions and corresponding responses happening during task solving.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model performance on the generated responses.\n\n[Environment Description] \nThe environment is called "Block World". \nThere is a 7DOF Franka Panda robot with a parallel gripper, it has a force sensor on the end effector. The robot is mounted on a table.\nThere are multiple blocks on tha table, here we use 2 blocks as example. The blocks are called {block A} and {block B}. \nThe blocks are initialized at a random position on the table. They have

 98%|█████████▊| 294/300 [2:38:51<04:30, 45.11s/it]

 98%|█████████▊| 295/300 [2:38:51<03:45, 45.11s/it]
Request 75 took 142.76s
Generated 2 instructions, kept 2 instructions
### [[{'role': 'user', 'content': 'You will be given a set of tasks in a robotic environment.\nYou are asked to simulate the task instructions and corresponding responses happening during task solving.\nSome of them are long-horizon tasks request multiple reasoning steps, so we are generating multi-turn instructions in a chain of thought way. \nThese task instructions will be given to a GPT model and we will evaluate the GPT model performance on the generated responses.\n\n[Environment Description] \nThe environment is called "Block World". \nThere is a 7DOF Franka Panda robot with a parallel gripper, it has a force sensor on the end effector. The robot is mounted on a table.\nThere are multiple blocks on tha table, here we use 2 blocks as example. The blocks are called {block A} and {block B}. \nThe blocks are initialized at a random position on the table. They hav

prompts: 100%|██████████| 1/1 [01:55<00:00, 115.96s/it]
301it [2:40:47, 32.05s/it]                         

301it [2:40:47, 48.73s/it]
Request 76 took 115.97s
Generated 6 instructions, kept 6 instructions


In [5]:
import utils
utils.full_inst_json_to_full_form_json('./gpt4_generation/cot_full_v3/cot_instrut_regen.json', 
                                       './gpt4_generation/cot_full_v3/cot_instrut_regen_formatted.json')

In [51]:
data_lists = list(open("./gpt4_generation_cot/cot_instrut_regen_gorilla.jsonl"))

In [52]:
data = data_lists[0]
data = json.loads(data)['code']
if '###Instruction.' in data:
    data = data.replace('###Instruction.', '###Instruction:')
if 'Output:' in data:
    a = {'instruction':data.split('Output:')[0].split('Instruction:')[1].split('###')[0],
        'input': data.split('Output:')[0].split('Input:')[1].split('###')[0],
        'output': data.split('Output:')[1]}

In [61]:
data.split('Output:')[1]

" {'explanation': No. We don't know the weight of the blocks, need to infer weight first., 'code': <nooutput>.}\n"